# Matching catalogs based on proximity (detailed)
Here we show the specific steps of matching two catalogs based on proximity

In [ ]:
%load_ext autoreload
%autoreload 2

## ClCatalogs
Given some input data

In [ ]:
import numpy as np
from astropy.table import Table
input1 = Table({ 
    'ID': [f'CL{i}' for i in range(5)],
    'RA': [0.0, 0.0001, 0.00011, 25, 20],
    'DEC': [0.0, 0.0, 0.0, 0.0, 0.0],
    'Z': [0.2, 0.3, 0.25, 0.4, 0.35],
    'MASS': [10**13.5, 10**13.4, 10**13.3, 10**13.8, 10**14],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0, 1.0],
})
input2 = Table({
    'ID': ['CL0', 'CL1', 'CL2', 'CL3'],
    'RA': [0.0, 0.0001, 0.00011, 25],
    'DEC': [0.0, 0, 0, 0],
    'Z': [0.3, 0.2, 0.25, 0.4],
    'MASS': [10**13.3, 10**13.4, 10**13.5, 10**13.8],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0],
})
display(input1)
display(input2)

Create two `ClCatalog` objects. For the proximity matching it is necessary to have at least (`ra, dec`). If the redshift is used in the matching, it also must be included. The same goes for the mass:

In [ ]:
from clevar.catalog import ClCatalog
c1 = ClCatalog('Cat1', id=input1['ID'], ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'])
c2 = ClCatalog('Cat2', id=input2['ID'], ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'])
# Format for nice display
for c in ('ra', 'dec', 'z'):
    c1[c].info.format = '.2f'
    c2[c].info.format = '.2f'
for c in ('mass',):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'
display(c1)
display(c2)

## Matching
Import the `ProximityMatch` and create a object for matching

In [ ]:
from clevar.match import ProximityMatch
mt = ProximityMatch()

### Prepare the catalogs
The first step is to prepare each catalog with the matching configuration:

- `delta_z`: Defines redshift window for matching. The possible values are:
  - `'cat'`: uses redshift properties of the catalog
  - `'spline.filename'`: interpolates data in `'filename'` assuming (z, zmin, zmax) format
  - `float`: uses `delta_z*(1+z)`
  - `None`: does not use z
- `match_radius`: Radius of the catalog to be used in the matching. If `'cat'` uses the radius in the catalog, else must be in format `'value unit'`. (ex: `'1 arcsec'`, `'1 Mpc'`)

In this case, because one of the configuraion radius has physical units, we also need a cosmology (`cosmo`) object to convert it to angular size (this is done internally).

In [ ]:
from clevar.cosmology import AstroPyCosmology
mt_config1 = {'delta_z':.2,
            'match_radius': '1 mpc',
            'cosmo':AstroPyCosmology()}
mt_config2 = {'delta_z':.2,
            'match_radius': '1 arcsec'}
mt.prep_cat_for_match(c1, **mt_config1)
mt.prep_cat_for_match(c2, **mt_config2)

This will add values to the `mt_input` attribute of the catalogs:

In [ ]:
display(c1.mt_input)
display(c2.mt_input)

### Multiple matching
The next step is to match the catalogs and store all candidates that pass the matching criteria. You can also pass the argument:
- `radius_selection`: Given a pair of clusters, which radius will be used for the matching.

In [ ]:
%%time
mt.multiple(c1, c2)
mt.multiple(c2, c1)

This will fill the `mt_multi_self` and `mt_multi_other` columns:

In [ ]:
display(c1)
display(c2)

### Unique matching
Once all candidates are stored in each catalog, we can find the best candidates. You can also pass the argument:
- `preference`: In cases where there are multiple matched, how the best candidate will be chosen.

In [ ]:
%%time
mt.unique(c1, c2, preference='angular_proximity')
mt.unique(c2, c1, preference='angular_proximity')

This will fill the `mt_self` and `mt_other` columns:

In [ ]:
display(c1)
display(c2)

### Cross matching
If you want to make sure the same pair was found in both directions:

In [ ]:
c1.cross_match()
c2.cross_match()

This will fill the `mt_cross` column:

In [ ]:
display(c1)
display(c2)

## Save and Load
The results of the matching can easily be saved and load using `ClEvaR` tools:

In [ ]:
mt.save_matches(c1, c2, out_dir='temp', overwrite=True)

In [ ]:
mt.load_matches(c1, c2, out_dir='temp')
display(c1)
display(c2)